In [7]:
from pathlib import Path
import pandas as pd

# --- parameters ---
folder = "E:\Archived\ACME_analysis"             # root folder to search (e.g. "/path/to/folder")
target_column = "zsipm_NE" # name of the column to analyze
pattern = "sequencedf_*.csv"

# --- collect stats ---
records = []
for f in Path(folder).resolve().rglob(pattern):
    try:
        df = pd.read_csv(f)
    except Exception as e:
        records.append({
            "file": str(f.resolve()),
            "mean": float("nan"),
            "std": float("nan"),
            "note": f"read_error: {e.__class__.__name__}"
        })
        continue

    if target_column not in df.columns:
        records.append({
            "file": str(f.resolve()),
            "mean": float("nan"),
            "std": float("nan"),
            "note": "missing_column"
        })
        continue

    s = pd.to_numeric(df[target_column], errors="coerce").dropna()
    if s.empty:
        records.append({
            "file": str(f.resolve()),
            "mean": float("nan"),
            "std": float("nan"),
            "note": "no_numeric_values"
        })
        continue

    records.append({
        "file": str(f.resolve()),
        "mean": s.mean(),
        "std": s.std(ddof=1),
        "note": ""
    })

summary_df = pd.DataFrame(records, columns=["file", "mean", "std", "note"])

# --- extract third-level folder name and last 9 chars ---
def get_third_layer_key(filepath: str):
    parts = Path(filepath).parts
    if len(parts) >= 4:
        third_layer = parts[3]  # adjust if root/drive prefix differs
        return third_layer[-9:]
    return ""

summary_df["sort_key"] = summary_df["file"].apply(get_third_layer_key)

# --- sort by the extracted key ---
summary_df = summary_df.sort_values("sort_key", ignore_index=True)

summary_df



,file,mean,std,note,sort_key
0,E:\Archived\ACME_analysis\noise0009.0133\Run R...,0.000609,0.000013,,0009.0133
1,E:\Archived\ACME_analysis\STIRAPwp0009.0267\Ru...,0.000593,0.000014,,0009.0267
2,E:\Archived\ACME_analysis\STIRAPwp0009.0269\Ru...,0.000541,0.000026,,0009.0269
3,E:\Archived\ACME_analysis\STIRAPwp0009.0278\Ru...,0.000559,0.000028,,0009.0278
4,E:\Archived\ACME_analysis\STIRAPwp0009.0282\Ru...,0.000518,0.000018,,0009.0282
...,...,...,...,...,...
882,E:\Archived\ACME_analysis\noise963\Run Results...,NaN,NaN,missing_column,noise963
883,E:\Archived\ACME_analysis\noise969\Run Results...,NaN,NaN,missing_column,noise969
884,E:\Archived\ACME_analysis\noise969\Run Results...,NaN,NaN,missing_column,noise969
885,E:\Archived\ACME_analysis\noise969\Run Results...,NaN,NaN,missing_column,noise969


In [8]:
summary_df.to_csv("summary2.csv", index=False)